# Шаблон архитектуры системы управления данными "Lakehouse"

## Мотивация

### Хранилища Данных - Data Warehouse, DWH

**Хранилища данных** (**Data Warehouse**) исторически были призваны решить задачу формирования аналитической картины для бизнеса на базе собираемой информации. Аналитика позволяет бинесу принимать взвешанные решения и формировать стратегию развития компании. Хранилища данных предъявляют требования к формату входящих данных, что позволяет пользователям хранилища данных работать с понятными данными. Характерной чертой традицонных хранилищ данных (**Data Warehouse**) является неразрывная связь между вычислительными ресурсами и ресурсами для хранения данных.

С увеличением объема поступающей информации традицонный подход к формированию хранилища данных столкнулись с большими проблемами: неразрывность слоёв хранения и обработки данных диктовала бизнесу вертикальное масштабирование. Бизнес был вынужен закупать железо, способное обслуживать пиковые нагрузки. Таким образом, бизнес тратил много денег, хотя пиковые нагрузки могут и не возникать часто.

В дополнение к увеличению объема обрабатываемой информации альтернативные форматы данных, которые не всегда имеют чёткую структуру (видео, музыка, текст и т.д.), также начали играть большую роль в построении аналитической картины. К неструктурированным данным **хранилища данных** были не готовы абсолютно.

### Озёра Данных - Data Lake

Для решения накопившихся проблем были разработаны платформы аналитики данных **второго поколения**.

Основная идея заключается в том, что все поступающие данные начали складываться в **озёра данных** (**Data Lake**) - кластер из дешёвых жестких дисков. Данные в **озёре данных** хранятся открытых (open source) форматах, таких как Apache Parquet или ORC.

Apache Hadoop с HDFS популяризровал подход с озером данных. От требования к формату входящих данных (`schema-on-write`) удалось перейти к `schema-on-read` - только конечный потребитель знает какие данные ему нужны, и он сам является ответственным за извлечение этих данных из озера данных. Таким образом, `ETL` (**Extract Transform Load**) был заменен на `ELT` (**Extract Load Transform**): бизнес смог собирать всю доступную информацию, которая потом анализировалась различными подходами.

Архитектура **Озеро Данных + Хранилища Данных** является доминирующей в современном мире.

Отличительной особенностью озера данных является разделение слоёв хранения данных и обработки данных: бизнес может заказывать дополнительные процессорные мощности при необходимости.

### Ограничения современной архитектуры

В свою очередь озера данных (Data Lake) принесли свои проблемы связанные с качеством и управлением данных. При этом данные по прежнему необходимо доставлять в хранилища данных, к которым подключены витрины данных. Таким образом, все поступающие данные проходят два или три шага:

- E**L**T для попадания в озеро данных (Data Lake),
- ET**L** для попадания в хранилище данных (Warehouse),
- (опиционально) ETL для машинного обучения.

Современний бизнес также полагается на системы машинного обучения и искуственного интеллекта, для которых ни озёра данных (Data Lake), ни хранилища данных (Data Warehouse) не являются идеальным решением.

Среди проблем современной архитектуры организации данных (**Озеро Данных** + **Хранилища Данных**) можно выделить следующие категории:

- **Надежность**,
- **Свежесть данных**,
- **Ограниченные возможности расширенной аналитики**,
- **Итоговая стоимость владения данными**.

#### Надежность

Задача поддежрки озера данных и хранилища данных в консистентном состоянии является сложной и затратной. Перекачка данных из озера данных в хранилище данных несёт риски связанные со сбоем оборудования или ошибками программиста, что может выражаться в некоторой разнице между озером данных и хранилищем данных, а это ухудшает качество данных.

#### Свежесть данных

Самые новые данные всегда сначала в озеро данных, а уже потом в хранилище. Перекачка данных из озера данных в хранилище не может выполниться мнгновенно. Таким образом, современные аналитические системы уступают своим предшественникам (**Data Warehouse**) по качеству свежести данных.

#### Ограниченные возможности расширенной аналитики

Бизнес должен иметь возможность задавать вопросы относительно будущего на основании собранных данных (например, "Кому эффективнее всего предоставить скидки?").

Современные фреймворки организации машинного обучения PyTorch, TensorFlow, XGBoost имеют ограниченные возможности работы на базе хранилища данных. В отличии от привычных хранилищам запросов BI характера, которые работают с небольшим объемом информации, фреймворки машинного обучения требуют обработки огромных объемов данных.

Для подключения машинного обучения есть два варианта:

- выгрузить данные из хранилища данных в файлы, которые можно использовать при работе с машинным обучением (третий ETL шаг),
- работа с озером данных напрямую. Это отключает возможности хранилища данных по управлению данными:
    - ACID транзакции,
    - версионирование данных,
    - индексирование.

#### Итоговая стоимость владения данными

Одновременная поддержка озера данных и хранилища данных фактически увеличивает затраты на хранение данных вдвое: одни и те же данные лежат как в озере данных ("грязные"), так и в хранилище данных ("чистые"). При этом также в два раза увеличиваются риски утечки данных.

### Требования к организации системы управления данными

Таким образом, необходимо организовать данные в озере данных, которое отвечает следующим требованиям:

1. открытость форматов данных (Parquet, ORC) для хранения как защита от Vendor Lock-In,
2. производительность сопоставимая с хранилищами данных,
3. возможности управления атрибутами/признаками (feature) данных, сопоставимые с хранилищами данных,
4. прямой `I/O` доступ к данным для расширенной аналитики.

Шаблон архитектуры **Lakehouse** позволяет построить озеро данных, которое отвечает поставленным требованиям.

## Шаблон Lakehouse

Lakehouse предлагает организовать систему управления данными на базе следующих концепций:

- хранение данных в кластера из дешевых жестких дисков (HDFS, S3, GCS),
- система демонстрирует функции аналитической СУБД:
    - ACID транзакции,
    - версионирование данных,
    - аудит,
    - индексы,
    - кеширование,
    - оптимизатор запросов.

Таким образом, система построеная по шаблону Lakehouse заключает в себе комбинацию ключевых преимуществ озёр данных (Data Lake) и хранилищ данных (Data Warehouse):

- низкая стоимость хранения данных,
- открытые форматы данных (как в Data Lake),
- мощные возможности управления данными и оптимизации запросов (как в Data Warehouse).

### Концепции Lakehouse

Шаблон Lakehouse состоит из следующих концепций:

- все данные хранятся в блочном хранилище вроде HDFS или S3;
- все данные приводятся к стандартному формату, например, Parquet;
- данные могут быть организованы в виде логических таблиц;
- поверх слоя хранения данных реализован слой метаданных;
- слой метаданных может содержать:
    - статистику по файлам,
    - список файлов, относящихся к текущей версии таблицы.
- ACID транзакции, версионирование, индексация - манипуляции со слоем метаданных;
- клиенты могут читать файлы из HDFS или S3 напрямую.

### Реализация Lakehouse

Форматы [Delta Lake](https://delta.io/) (Databricks), [Apache Iceberg](https://iceberg.apache.org/) (Netflix) и [Apache Hudi](https://hudi.apache.org/) (Uber) реализуют концепции похожие на Lakehouse схожим образом:

- данные сохраняются в формате Parquet или ORC в блочное хранилище (HDFS, S3),
- сохраненные файлы неизменны,
- рядом с сохраненными данными создаеются файлы с метаданными,
- при внесении изменений в файлы, создаются новые файлы,
- метаданные отслеживают какие из имеющихся файлов относятся к текущей версии.

Формат Delta Lake был разработан в компании Databricks, которая развивает Apache Spark, поэтому построение системы управления данными на примере Lakehouse в практической части будет основываться на Delta Lake.

> **Delta Lake - основной формат для построения Lakehouse!**

### Ограничения Delta Lake

Один файл с метаданными относится к одному Parquet файлу с данными, поэтому, к сожалению, невозможно получить ACID транзакции, включающие множество файлов. Эта проблема известна и находится на контроле ответственных организаций.

Учитывая, что Databrics запускает примерно 50% (на 2021 год) всей нагрузки своих клиентов через Delta Lake, возможно отсутствие ACID транзакций на несколько файлов не является большой проблемой.

### Особенности Delta Lake

Вне зависимости от используемого блочного хранилища Delta Lake эффективно реализует оптимизации:

- пропуск файлов при сканировании,
- кластеризация строк.

#### Анализ статистики для пропуска файлов при сканировании

Вне зависимости от используемого блочного хранилища, файл с метаданными хранит информацию, которая может использоваться оптимизатором запросов. В метаданных хранится статистика о максимальных и минимальных значениях колонок для каждого файла, что позволяет исключать файлы из сканирования.

#### Пропуска файлов при сканировании на базе Bloom фильтра

Для каждого parquet файла строится Bloom фильтр, который также позволяет эффективно пропускать файлы, в которых точно нет требуемых данных.

#### Кластеризация строк

Delta Lake использует продвинутые алгоритмы для определения наилучшей организации строки на диске, с учетом, что несколько строк будут наиболее часто считываться вместе.

# Запуск Spark

## Конфигурация

Для работы с Delta Lake из Python необходимо установить пакет `delta-lake`:

In [ ]:
! pip install delta-spark==3.0.0

### Запуск Apache Spark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col
from pyspark.sql import functions as F

In [ ]:
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable

In [ ]:
builder = (
    SparkSession
        .builder
        .appName("Delta Demo")
        .master("local[4]")
        # .config("spark.jars.packages", "io.delta:delta-core_2.12:2.1.0")
        .config("spark.sql.warehouse.dir", "data/spark-warehouse")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)

Функция `configure_spark_with_delta_pip` активиурет возможности Delta Lake:

In [ ]:
spark = configure_spark_with_delta_pip(builder) \
    .getOrCreate()

sc = spark.sparkContext

In [ ]:
print(configure_spark_with_delta_pip.__doc__)

### Подготовка данных

Распаковать подготовленные архивы с данными:

In [ ]:
! cd /tmp && rm -rf steam taxi.parquet && unzip -o ~/work/data/steam.zip && unzip -o ~/work/data/taxi.zip

Функция `dump_parquet` скроет подробности созхранения датафрейма на диск как parquet:

In [ ]:
def dump_parquet(df: DataFrame, num: int, loc: str):
    (df
        .repartition(num)
        .write
        .mode("overwrite")
        .parquet(loc)
    )

Файлы будут располагаться в HDFS, поэтому необходимо очистить файловую систему:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -rm -f -r /user/jovyan/data && \
hdfs dfs -mkdir -p /user/jovyan/data/stream && \
hdfs dfs -mkdir -p /user/jovyan/data/taxi

Загрузить локальные файлы в Apache Spark, разбить их на 4 партиции и положить в HDFS:

In [ ]:
sc.setJobDescription("Разбить датафреймы steam на 4 партиции")

# dump_parquet(spark.read.parquet("file:///tmp/steam/details.parquet"), 4, "/user/jovyan/data/steam/details")
# dump_parquet(spark.read.parquet("file:///tmp/steam/tags.parquet"), 4, "/user/jovyan/data/steam/tags")
dump_parquet(spark.read.parquet("file:///tmp/steam/games.parquet"), 4, "/user/jovyan/data/steam/games")
# dump_parquet(spark.read.parquet("file:///tmp/taxi.parquet"), 4, "/user/jovyan/data/taxi")

Файлы загружены в HDFS:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /user/jovyan/data

## Delta таблицы

In [ ]:
my_first_table_path = 'data/my-first-delta-table'

### Создание Delta таблицы

Сохранение датафрейма в формате `delta` создает новую Delta таблицу:

In [ ]:
data = spark.range(0, 5)
(
    data
        .write
        .format("delta")
        .save(my_first_table_path)
)

### Загрузка Delta таблицы

Чтение данных в формате `delta` читает данные из таблицы:

In [ ]:
df = (
    spark
        .read
        .format("delta")
        .load(my_first_table_path)
)
df.show()

### Обновление Delta таблицы

Запись датафрейма в формате `delta` и в режиме `overwrite` обновляет данные в `delta` таблице:

In [ ]:
df = spark.range(5, 10)
(
    df
        .write
        .format("delta")
        .mode("overwrite")
        .save(my_first_table_path)
)

In [ ]:
df = (
    spark
        .read
        .format("delta")
        .load(my_first_table_path)
)
df.show()

### Частичное обновление Delta таблицы

Данные в таблице можно обновить по условию:

In [ ]:
deltaTable = DeltaTable.forPath(spark, my_first_table_path)

deltaTable.update(
    condition = F.expr("id % 2 == 0"),
    set = { "id": F.expr("id + 100") }
)

In [ ]:
df = (
    spark
        .read
        .format("delta")
        .load(my_first_table_path)
)
df.show()

### Удаление данных Delta таблицы

Удалить данные в таблице можно можно обновить по условию:

In [ ]:
deltaTable = DeltaTable.forPath(spark, my_first_table_path)
deltaTable.delete(condition = F.expr("id % 2 == 0"))

In [ ]:
df = (
    spark
        .read
        .format("delta")
        .load(my_first_table_path)
)
df.show()

### Слияние данных - Merge, Upsert

Данные могут быть записаны в таблицу по принципу слияния (merge, upsert), т.е. существующие данные заменяются, новые данные добавляются:

In [ ]:
deltaTable = DeltaTable.forPath(spark, my_first_table_path)

# Upsert (merge) new data
newData = spark.range(0, 20).withColumnRenamed("id", "x")

deltaTable \
  .merge(newData, col("id") == col("x")) \
  .whenMatchedUpdate(set = { "id": col("x") }) \
  .whenNotMatchedInsert(values = { "id": col("x") }) \
  .execute()

In [ ]:
df = (
    spark
        .read
        .format("delta")
        .load(my_first_table_path)
)
df.show(5)

Обратите внимание, что операция `execute` выполняется немедленно.

## Машина времени

Все файлы, загруженные в HDFS через Delta Lake, становятся неизменяемыми. Но как тогда выполняется обновление данных?

Delta создает новые файлы в директории таблицы и при помощи дополнительных файлов отслеживает, какие файлы составляют содержимое текущей версии таблицы.

Delta поддерживает версионирование, а значит в любой момент можно вернуться к любой из предыдущих версий: файлы по прежнему находятся в директории таблицы. В качестве примера можно вернуться к самой первой версии:

In [ ]:
df = (
    spark.read
        .format("delta")
        .option("versionAsOf", 0)
        .load(my_first_table_path)
)

df.show()

## Конвертация Parquet в Delta Lake

Delta Lake работает на базе Parquet, поэтому конвертация выполняется простым добавлением файлов с метаданными в директорию с Parquet файлами:

Состояние директории без файлов с метаданными Delta Lake:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /user/jovyan/data/taxi

In [ ]:
taxi_path = '/user/jovyan/data/taxi'

In [ ]:
deltaTable = DeltaTable.convertToDelta(spark, f"parquet.`{taxi_path}`")

In [ ]:
print(DeltaTable.convertToDelta.__doc__)

Состояние директории после конвертации в Delta Lake:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /user/jovyan/data/taxi

Директория `_delta_log` содержит метаданные, в частности, схему:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -cat /user/jovyan/data/taxi/_delta_log/00000000000000000000.json | \
grep metaData | json_pp | \
grep schema | sed 's,\s*"schemaString" : ",,;s,"$,,;s,\\",",g' | json_pp > schema.json

Вывод команды сохранен в [schema.json](schema.json)

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -cat /user/jovyan/data/taxi/_delta_log/00000000000000000000.json | \
grep add |& head -n 1 | json_pp | \
grep stats | sed 's,\s*"stats" : ",,;s,"$,,;s,\\",",g' | json_pp > stats.json

Вывод команды сохранен в [stats.json](stats.json)

## Z-Order оптимизация

After you are done with the writes on the delta table, it might be useful to call optimize on it.

- call [optimize](https://docs.delta.io/latest/api/python/index.html#delta.tables.DeltaTable.optimize)
- use [z-order](https://docs.delta.io/latest/api/python/index.html#delta.tables.DeltaOptimizeBuilder.executeZOrderBy) by the column user_id
- check manually the files under the table to see that it was compacted


In [ ]:
deltaTable = DeltaTable.forPath(spark, taxi_path)

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /user/jovyan/data/taxi

In [ ]:
df = deltaTable.optimize().executeZOrderBy("passenger_count")

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /user/jovyan/data/taxi

In [ ]:
df.printSchema()

In [ ]:
df.select("metrics.zOrderStats").show(5, False)

In [ ]:
df.explain()

In [ ]:
(
deltaTable.history()
    .select(
        "version",
        "timestamp",
        "operation",
        "operationMetrics.numRemovedFiles",
        "operationMetrics.numAddedFiles",
        "operationMetrics.numConvertedFiles")
    .show(5, False)
)

## Вакуумная очистка

Количество файлов таблицы может расти бесконтрольно, но не все из них нужны. Файлы, которые относятся к старым версиям можно удалять при помощи [`vacuum`](https://docs.delta.io/latest/api/python/index.html#delta.tables.DeltaTable.vacuum):

In [ ]:
deltaTable = DeltaTable.forPath(spark, taxi_path)

In [ ]:
import sys

try:
    deltaTable.vacuum(0)
except Exception as e:
    print(e, file=sys.stderr)

In [ ]:
spark.conf.set('spark.databricks.delta.retentionDurationCheck.enabled', False)

In [ ]:
deltaTable.vacuum(0)

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /user/jovyan/data/taxi

In [ ]:
deltaTable.history().show()

In [ ]:
import sys

try:
    (
    spark.read
        .format("delta")
        .option("versionAsOf", 0)
        .load(taxi_path)
        .show()
    )
except Exception as e:
    print(e.java_exception.getMessage(), file=sys.stderr)

## Построение Lakehouse архитектуры

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col
from pyspark.sql import functions as F

In [ ]:
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable

In [ ]:
spark.stop()

In [ ]:
builder = (
    SparkSession
        .builder
        .appName("Delta Demo")
        .master("local[4]")
        .config("spark.sql.warehouse.dir", "data/spark-warehouse")
        .config("spark.driver.memory", "4g")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .enableHiveSupport()
)

spark = configure_spark_with_delta_pip(builder) \
    .getOrCreate()

sc = spark.sparkContext

In [ ]:
games_path = "data/steam/games"
games_delta = DeltaTable.convertToDelta(spark, f"parquet.`{games_path}`")

In [ ]:
from random import randrange

ids = [ F.lit(x) for x in range(1, 10) ]

reviews_metacritic = (games_delta.toDF()
    .withColumn("id", F.explode(F.array(ids)))
    .select(
        F.expr("app_id * 1000000 + id").alias("id"),
        "app_id",
        F.expr("STRING(DATE_ADD(date_release, CAST(floor(rand() * 500) AS INT)))").alias("review_date"),
        col("title").alias("review"),
        F.floor(F.rand() * 100).alias("rating"),
        F.floor(F.rand() * 100000).alias("user_id"),
    )
)
reviews_metacritic.show(5, False)

In [ ]:
(
reviews_metacritic
    .write
    .format("delta")
    .partitionBy("rating")
    .mode("overwrite")
    .saveAsTable("reviews_metacritic")
)

In [ ]:
from random import randrange

ids = [ F.lit(x) for x in range(1, 5) ]

reviews_igromania = (games_delta.toDF()
    .withColumn("id", F.explode(F.array(ids)))
    .select(
        F.expr("app_id * 1000000 + id").alias("id"),
        "app_id",
        F.expr("unix_timestamp() + floor(rand() * 200)").alias("review_date_unix"),
        col("title").alias("review"),
        F.round(F.rand() * 10, 1).alias("rating"),
        F.floor(F.rand() * 100000).alias("user_id"),
    )
)
reviews_igromania.show(5, False)

In [ ]:
(
reviews_igromania
    .write
    .format("delta")
    .partitionBy("rating")
    .mode("overwrite")
    .saveAsTable("reviews_igromania")
)

In [ ]:
spark.table("reviews_igromania").groupBy("id").count().where("count > 1").show()
spark.table("reviews_metacritic").groupBy("id").count().where("count > 1").show()

In [ ]:
users_df = (
    spark.range(100000)
        .withColumn("user_id", F.concat(F.lit("user"), F.lit("_"), col("id")))
        .withColumn("gender", F.when(F.rand() > 0.6, "M").otherwise("F"))
        .withColumn("dob", F.expr("ADD_MONTHS(current_date(), -CAST(floor(rand() * 25 * 12) AS INT) -10 * 12)"))
)
users_df.show(5)

In [ ]:
(
users_df
    .write
    .format("delta")
    .partitionBy("gender")
    .mode("overwrite")
    .saveAsTable("users")
)

In [ ]:
# DeltaTable.forName(spark, "reviews_igromania").optimize().executeZOrderBy("app_id")
# DeltaTable.forName(spark, "reviews_metacritic").optimize().executeZOrderBy("app_id")
# DeltaTable.forPath(spark, "data/steam/games").optimize().executeZOrderBy("app_id")

### Серебрянный слой

In [ ]:
# spark.sql("""
# CREATE OR REPLACE TABLE reviews
# USING DELTA
# PARTITIONED BY (source, rating)
# --ZORDER BY  app_id
# AS (
#   SELECT id
#        , app_id
#        , CAST(from_unixtime(review_date_unix) AS TIMESTAMP) review_date
#        , review
#        , ROUND(rating * 100, 0) rating
#        , user_id
#        , 'IGROMANIA' source
#     FROM reviews_igromania
#    WHERE 1 != 1

#   UNION ALL

#   SELECT id
#        , app_id
#        , to_timestamp(review_date) review_date
#        , review
#        , ROUND(rating, 0) rating
#        , user_id
#        , 'METACRITIC' source
#     FROM reviews_metacritic
#    WHERE 1 != 1
# )
# """)

In [ ]:
spark.sql("DROP TABLE IF EXISTS reviews_silver")

In [ ]:
spark.sql("""
CREATE OR REPLACE TABLE reviews_silver (
    id STRING NOT NULL,
    app_id INT NOT NULL,
    review_ts TIMESTAMP NOT NULL,
    review STRING,
    rating INT,
    user_id INT NOT NULL,
    source STRING NOT NULL
)
USING DELTA
PARTITIONED BY (source, rating)
""")

In [ ]:
spark.sql("DESC EXTENDED reviews_silver").show()

In [ ]:
reviews_silver = DeltaTable.forName(spark, "reviews_silver")

In [ ]:
# reviews_bronze_igromania = spark.table("reviews_igromania").withColumn("id", F.concat(F.lit("igromania_"), col("id")))
# reviews_bronze_metacritic = spark.table("reviews_metacritic").withColumn("id", F.concat(F.lit("metacritic_"), col("id")))

In [ ]:
reviews_bronze_igromania = spark.sql("""
SELECT 'IGROMANIA' || id id
     , app_id
     , CAST(from_unixtime(review_date_unix) AS TIMESTAMP) review_ts
     , review
     , ROUND(rating * 10, 0) rating
     , user_id
     , 'IGROMANIA' source
  FROM reviews_igromania
""")

In [ ]:
reviews_bronze_metacritic = spark.sql("""
SELECT 'METACRITIC' || id id
     , app_id
     , to_timestamp(review_date) review_ts
     , review
     , ROUND(rating, 0) rating
     , user_id
     , 'METACRITIC' source
  FROM reviews_metacritic
""")

In [ ]:
(
reviews_silver.alias("silver")
  .merge(reviews_bronze_igromania.alias("bronze"), col("silver.id") == col("bronze.id"))
  .whenMatchedUpdateAll()
  .whenNotMatchedInsertAll()
  .execute()
)

In [ ]:
(
reviews_silver.alias("silver")
  .merge(reviews_bronze_metacritic.alias("bronze"), col("silver.id") == col("bronze.id"))
  .whenMatchedUpdateAll()
  .whenNotMatchedInsertAll()
  .execute()
)

In [ ]:
spark.table("reviews_silver").count()

In [ ]:
spark.table("reviews_silver").groupBy("id").count().where("count > 1").show()

In [ ]:
# reviews_silver.optimize().executeZOrderBy("app_id")

### Золотой слой - Golden Layer

In [ ]:
spark.sql("DROP TABLE IF EXISTS game_review_facts_golden")

In [ ]:
spark.sql("""
CREATE OR REPLACE TABLE game_review_facts_golden (
    user_id INT NOT NULL,
    app_id INT NOT NULL,
    review_id STRING NOT NULL,
    ts TIMESTAMP NOT NULL
)
USING DELTA
""")

In [ ]:
golden_facts_stream = (
    spark.readStream
        .format("delta")
        # .table("reviews_silver")
        .load("/user/jovyan/data/spark-warehouse/reviews_silver")    
        .writeStream
        .format("console")
)
golden_facts_stream.start()

In [ ]:
spark.range(10).show()

In [ ]:
(
game_review_facts_golden.alias("gold")
  .merge(reviews_silver.alias("silver"), col("gold.user_id") == col("bronze.id"))
  .whenMatchedUpdateAll()
  .whenNotMatchedInsertAll()
  .execute()
)

In [ ]:
taxi_df = spark.read.parquet("/user/jovyan/data/taxi")

In [ ]:
taxi_df = spark.read.parquet("/user/jovyan/data/taxi")
taxi = bucketing(taxi_df, "taxi", "passenger_count")

In [ ]:
spark.sql("""
CREATE TABLE IF NOT EXISTS taxi
USING DELTA
PARTITIONED BY (passenger_count)
""").collect()

In [ ]:
spark.catalog.listDatabases()

In [ ]:
spark.sql("DROP TABLE IF EXISTS taxi").collect()